In [11]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline

In [13]:
# '네이버 영화 리뷰 데이터'로 검색

train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [14]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

In [15]:
# 트레인 데이터 셋 
# 중복 여부 확인 
train_df.document.nunique()

146182

In [16]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape 

(146183, 3)

In [17]:
# null 데이터가 있는지 확인
train_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [18]:
# null 데이터 제거
train_df = train_df.dropna(how='any')
train_df.shape 

(146182, 3)

In [19]:
# 긍정 , 부정 레이블 분포 
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트 데이터 셋에도 적용 


In [20]:
# 중복 여부 확인
test_df.document.nunique()

49157

In [21]:
# 중복 데이터 제거
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape 

(49158, 3)

In [22]:
# null 데이터가 있는지 확인
test_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [23]:
# null 데이터 제거
test_df = test_df.dropna(how='any')
test_df.shape 

(49157, 3)

In [24]:
# 긍정 , 부정 레이블 분포 
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

In [ ]:
### 2.텍스트 전처리 

In [26]:
import warnings
warnings.filterwarnings(action='ignore')
# 이게 오류 메시지 무시하는 거구나! 

In [27]:
# 한글과 공백 이외는 제거 
train_df['document'] = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [28]:
# '' 데이터는 Nan으로 변환 후 제거
train_df['document'].replace('', np.nan, inplace=True)
train_df.document.isnull().sum()

391

In [30]:
train_df = train_df.dropna(how='any')
train_df.shape

(145791, 3)

In [ ]:
- test dataset 

In [32]:
test_df['document'] = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
test_df['document'].replace('', np.nan, inplace=True)
test_df = test_df.dropna(how='any')
test_df.shape


(48995, 3)

In [33]:
train_df.to_csv('naver_movie_train.tsv', sep='\t')
test_df.to_csv('naver_movie_test.tsv', sep='\t')

In [ ]:
## 3 .한글 처리 

In [34]:
from konlpy.tag import Okt
okt = Okt()

In [35]:
stopwords = ['의' ,'가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다', '을','ㅋㅋ','ㅠㅠ','ㅎㅎ']

In [36]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text)

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [37]:
okt.morphs(text, stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [38]:
from tqdm import tqdm_notebook
X_train = []
for sentence in tqdm_notebook(train_df.document):
    morphs = okt.morphs(sentence, stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    X_train.append(temp_X)

In [39]:
X_test = []
for sentence in tqdm_notebook(test_df.document):
    morphs = okt.morphs(sentence, stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    X_test.append(temp_X)

In [41]:
y_train = train_df.label.values
y_test = test_df.label.values

### 4. Feature 변환 모델 학습/예측/평가

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 

In [43]:
cvect = CountVectorizer()
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)

In [44]:
lr = LogisticRegression()
lr.fit(X_train_cv, y_train)

LogisticRegression()

In [45]:
pred = lr.predict(X_test_cv)
accuracy_score(y_test, pred)

0.8253699357077253

- 5. 실제 테스트

In [47]:
review1 = '1편에 비해 스케일도 크고 간만에 긴장감있는 영화를 영화관에서 봐서 재미있었다 펜데믹 상황에서의 재난물이라 더욱 ㅠㅠㅠ'
review2 = '돈이 아까운 쓰레기 영화!!! \^^/'

In [48]:
review1 = review1.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
review1

'1편에 비해 스케일도 크고 간만에 긴장감있는 영화를 영화관에서 봐서 재미있었다 펜데믹 상황에서의 재난물이라 더욱 ㅠㅠㅠ'

In [49]:
import re 
review1 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', review1)
review1

'편에 비해 스케일도 크고 간만에 긴장감있는 영화를 영화관에서 봐서 재미있었다 펜데믹 상황에서의 재난물이라 더욱 ㅠㅠㅠ'

In [50]:
morphs = okt.morphs(review1) 
review = ' '.join([word for word in morphs if not word in stopwords])
review 


'편 비해 스케일 크고 간만 긴장감 있는 영화 영화관 에서 봐서 재미있었다 펜데믹 상황 에서의 재난 물이 라 더욱 ㅠㅠㅠ'

In [52]:
review_cv = cvect.transform([review])
pred = lr.predict(review_cv)
pred[0]

1

In [55]:
review2 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', review2)
morphs = okt.morphs(review2) 
review = ' '.join([word for word in morphs if not word in stopwords])
review_cv = cvect.transform([review])
pred = lr.predict(review_cv)
pred[0]

1

In [56]:
reviews = ['1편에 비해 스케일도 크고 간만에 긴장감있는 영화를 영화관에서 봐서 재미있었다 펜데믹 상황에서의 재난물이라 더욱 ㅠㅠㅠ','돈이 아까운 쓰레기 영화!!! \^^/']





In [57]:
review_list = []
for review in reviews:
    review = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', review)
    morphs = okt.morphs(sentence, stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    review_list.append(temp_X)

In [58]:
review_list

['마무리 또 왜 이래', '마무리 또 왜 이래']

In [59]:
review_cv = cvect.transform(review_list)
pred = lr.predict(review_cv)
pred

array([0, 0], dtype=int64)

### 6. Grid SearchCV 로 최적 파라미터 찾기 

In [60]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


In [61]:
pipeline = Pipeline([
    ('cvect', CountVectorizer()),
    ('lr', LogisticRegression())
])
params = {
    'cvect__ngram_range': [(1,1),(1,2)],
    'cvect__max_df': [0.9, 0.99],
    'cvect__min_df': [1,3],
    'lr__C': [1,5]
}

In [62]:
grid_pipe = GridSearchCV(
    pipeline, param_grid=params, cv=3, scoring='accuracy', n_jobs=-1

)
%time grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_score_, grid_pipe.best_params_)


Wall time: 2min 22s
0.840538853564349 {'cvect__max_df': 0.9, 'cvect__min_df': 1, 'cvect__ngram_range': (1, 2), 'lr__C': 1}


In [66]:
pred = grid_pipe.best_estimator_.predict(X_test)
acc = accuracy_score(y_test, pred)
print(f'CounterVectorizer + LogisticRegression 정확도: {acc:.4f}')

CounterVectorizer + LogisticRegression 정확도: 0.8462
